# __Task 1__

Давайте уточним правило трёх сигм. Утверждение: 99.7% вероятностной массы случайной величины $X\sim N\left(\mu,\sigma^2\right)$ лежит в интервале $\mu\pm c \cdot \sigma$. Чему равно точное значение константы c? Округлите ответ до четырёх знаков после десятичной точки. 

In [1]:
import scipy.stats
import numpy as np
print(round(scipy.stats.norm.ppf(0.9985),4))

2.9677


In [2]:
scipy.stats.norm.ppf?

# __Task 2__

Пусть $X\sim N\left(\mu, \sigma^2\right)$. Какое распределение имеет величина $\frac{\bar{X}_n-\mu}{S_n/\sqrt{n}}$

Стьюдента

# __Task 5__

В пятилетнем рандомизированном исследовании Гарвардской медицинской школы 11037 испытуемых через день принимали аспирин, а ещё 11034 — плацебо. Исследование было слепым, то есть, испытуемые не знали, что именно они принимают.

За 5 лет инфаркт случился у 104 испытуемых, принимавших аспирин, и у 189 принимавших плацебо. 

Оцените, насколько вероятность инфаркта снижается при приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки.

### Отметим, что все пациенты, участвовашие в эксперименте не знают друг о друге, таким образом события являются независимыми.
Чтобы выяснить, какая из долей лучше, требуется рассчитать доверительный интервал на разность двух этих долей.
Будем сравнивать с первой долей

In [26]:
# количество, принимавших аспирин
n_1 = 11037
# количество, принимавших плацебо
n_2 = 11034
# количество умерших, принимавших аспирин
k_1 = 104
# количество умерших, принимавших плацебо
k_2 = 189

# доля(вероятность) умереть при этом принимать аспирин
p_1 = k_1 / n_1
# доля(вероятность) умереть при этом принимать плацебо
p_2 = k_2 / n_2
p_1,p_2
print(round((p_2 - p_1),5))

0.00771


# __Task 6__

Постройте теперь 95% доверительный интервал для снижения вероятности инфаркта при приёме аспирина. Чему равна его верхняя граница? Округлите ответ до четырёх знаков после десятичной точки.  

In [21]:
def proportions_confint_diff_ind(n_1, k_1, n_2, k_2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(k_1) / n_1
    p2 = float(k_2) / n_2
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n_1 + p2 * (1 - p2)/ n_2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n_1 + p2 * (1 - p2)/ n_2)
    
    return (round(left_boundary,5), round(right_boundary,5))

In [22]:
print("confidence interval: [%f, %f]" % proportions_confint_diff_ind(n_1, k_1, n_2, k_2))

confidence interval: [-0.010720, -0.004690]


# __Task 7__

Продолжим анализировать данные эксперимента Гарвардской медицинской школы. 

Для бернуллиевских случайных величин $X\sim Ber(p)X∼Ber(p)$ часто вычисляют величину $\frac{p}{1-p}$, которая называется шансами (odds). Чтобы оценить шансы по выборке, вместо p нужно подставить $\hat{p}$
Например, шансы инфаркта в контрольной группе, принимавшей плацебо, можно оценить как 
$\frac{\frac{189}{11034}}{1-\frac{189}{11034}} = \frac{189}{11034-189}\approx 0.0174$


Оцените, во сколько раз понижаются шансы инфаркта при регулярном приёме аспирина. Округлите ответ до четырёх знаков после десятичной точки. 

In [34]:
odds1 = p_1 / (1 - p_1)
odds2 = p_2 / (1 - p_2)
print(odds1, odds2)
print(round((odds2 / odds1), 4))

0.009512485136741973 0.017427385892116183
1.8321


# __Task 8__

Вопрос 8
Величина, которую вы оценили в предыдущем вопросе, называется отношением шансов. Постройте для отношения шансов 95% доверительный интервал с помощью бутстрепа. Чему равна его нижняя граница? Округлите ответ до 4 знаков после десятичной точки.

Чтобы получить в точности такой же доверительный интервал, как у нас:

составьте векторы исходов в контрольной и тестовой выборках так, чтобы в начале шли все единицы, а потом все нули;
установите random seed=0;
сделайте по 1000 псевдовыборок из каждой группы пациентов с помощью функции get_bootstrap_samples.

In [49]:
np.random.randint?

In [35]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [36]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [37]:
def get_p(k,n):
    return float(k)/n

In [38]:
def get_odds(p):
    return p/(1-p)

In [39]:
def get_chances_relation(sample1, sample2):
    return get_odds(get_p(sum(sample2), len(sample2)))/get_odds(get_p(sum(sample1), len(sample1)))

In [41]:
data1 = np.zeros(n_1)
data1[:k_1] = 1
data2 = np.zeros(n_2)
data2[:k_2] = 1

In [43]:
np.random.seed(0)
samples1 = get_bootstrap_samples(data1, 1000)
samples2 = get_bootstrap_samples(data2, 1000)
rel_scores = []
for i in range(1000):
    rel_scores.append(get_chances_relation(samples1[i], samples2[i]))

In [45]:
samples1

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [46]:
samples2

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [44]:
print("95% confidence interval:",  stat_intervals(rel_scores, 0.05))

95% confidence interval: [1.44419465 2.34321168]
